In [40]:
# %pip install nltk sklearn
import nltk
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize   
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
import nltk
from scipy.sparse import hstack
from sklearn.preprocessing import normalize

wordnet_lemma  = WordNetLemmatizer()

clf = MultinomialNB()

def lemmatize(word, pos='v'):
    return wordnet_lemma.lemmatize(word, pos=pos)

stop = stopwords.words('english')

def pre_process(text):
    text = re.sub('<.*?>', ' ', text) # Remove html tags. TODO: Check for relevant information inside?
    text = re.sub(r'=\d\w',' ',text) # Remove encoded symbols like =2E, =3A....    
    return text

def tokenize_lemmatize(text):
    out = word_tokenize(text)
    out = [lemmatize(word,
#                      pos="v"
                    ) for word in out]
    out = [word for word in out if word not in stop]
    return out

data = pd.read_csv("data/kg_train.csv",encoding='latin-1')#.sample(1000)

# CLEAN DATA
data['processed_text'] = data['text'].apply(pre_process)

# Transform Data
# Change vectorizer for testing if required
vectorizer = TfidfVectorizer(ngram_range=(1,4), tokenizer=tokenize_lemmatize)
# vectorizer = CountVectorizer(tokenizer=tokenize_lemmatize, ngram_range=(1,4))

X = vectorizer.fit_transform(data['processed_text'])
X = hstack((X, normalize(scipy.sparse.csc_matrix((data['text'].apply(len))).T)))
Y = data['label']

# Evaluate
print(cross_validate(clf, X, Y, n_jobs=4, cv=5)['test_score'])

# Train
clf.fit(X, Y)


[0.98575021 0.99077955 0.98575021 0.98575021 0.98825503]


MultinomialNB()

In [41]:
data['prediction'] = clf.predict(X)
data[data['label']!=data['prediction']]

,text,label,processed_text,prediction
125,<HTML><BODY><div><div><TABLE cellSpacing=0 cel...,1,,0
305,--,0,--,1
765,,1,,0
790,<html><div style='background-color:'><DIV><DIV...,1,,0
1188,Your friend,0,Your friend,1
1642,A),0,A),1
2068,-----,0,-----,1
2737,<DIV id=3Dhotbar_promo></DIV></TD></TR></TBODY...,1,,0
2832,<DIV id=3Dhotbar_promo></DIV></TD></TR></TBODY...,1,,0
2848,Flood Statement Randolph (Arkansas) FLOOD STAT...,1,Flood Statement Randolph (Arkansas) FLOOD STAT...,0


In [6]:
test_data = pd.read_csv("data/kg_test.csv",encoding='latin-1')
test_data['processed_text'] = test_data['text'].apply(pre_process)
X_test = vectorizer.transform(test_data['processed_text'])


In [7]:
clf = MultinomialNB()

clf.fit(X, Y)
result = clf.predict(X_test)

df = pd.DataFrame(result, columns=['Category'])
df['Id'] = df.index
df = df[['Id','Category']]
df.to_csv('prediction.csv2', index=False)



In [ ]:
# Uncomment for submission
# !kaggle competitions submit -c dsub-fraudulentemails -f prediction.csv -m "Message"
